In [1]:
import pandas as pd

In [2]:
fgk = pd.read_csv('data/fgk.txt', sep="\t")
fgk.head()

,star,group,ID,ID_alt,star_alt1,star_alt2,origin,snr,R,Rmax,...,e[Ti 1/H],n[Ti 1/H],[Ti 2/H],e[Ti 2/H],n[Ti 2/H],[V 1/H],e[V 1/H],n[V 1/H],n_spectra,flag
0,HIP101345,G Subgiant (IV),HD195564_HAR_1,HIP101345_HARPS_1,HD195564,-,HARPS,902,42000,115000,...,0.023,34,0.062,0.027,7,-0.121,0.024,27,2,-
1,HIP101345,G Subgiant (IV),HD195564_NAR_1,HIP101345_NARVAL_1,HD195564,-,NARVAL,369,42000,68000,...,0.023,34,0.062,0.027,7,-0.121,0.024,27,2,-
2,HIP10234,K Giant (III),HD13468_FER_1,HIP10234_FEROS_1,HD13468,-,FEROS,121,42000,48000,...,0.059,36,-0.221,0.042,8,-0.532,0.061,29,2,-
3,HIP10234,K Giant (III),HD13468_HAR_1,HIP10234_HARPS_1,HD13468,-,HARPS,95,42000,115000,...,0.059,36,-0.221,0.042,8,-0.532,0.061,29,2,-
4,HIP102422,K Subgiant (IV),HD198149_NAR_1,HIP102422_NARVAL_1,HD198149,-,NARVAL,908,42000,68000,...,0.062,35,-0.218,0.031,8,-0.447,0.047,30,2,-


In [3]:
fgk[fgk.origin=='UVES'].head()

,star,group,ID,ID_alt,star_alt1,star_alt2,origin,snr,R,Rmax,...,e[Ti 1/H],n[Ti 1/H],[Ti 2/H],e[Ti 2/H],n[Ti 2/H],[V 1/H],e[V 1/H],n[V 1/H],n_spectra,flag
25,HIP112440,K Giant (III),HD215665_UVE_1,HIP112440_UVES_1,HD215665,-,UVES,353,42000,45254,...,0.058,33,-0.040,0.054,10,-0.094,0.043,38,3,-
34,HIP113357,G Dwarf (V),HD217014_UVE_1,HIP113357_UVES_1,HD217014,-,UVES,1158,42000,107200,...,0.019,90,0.190,0.018,19,0.159,0.019,74,5,-
38,HIP114855,K Giant (III),HD219449_UVE_1,HIP114855_UVES_1,HD219449,-,UVES,524,42000,74450,...,0.044,42,-0.100,0.064,11,-0.108,0.077,47,3,-
43,HIP115227,K Giant (III),HD220009_UVE_1,HIP115227_UVES_1,HD220009,HD220009,UVES,241,42000,49505,...,0.082,42,-0.466,0.028,12,-0.748,0.087,39,3,-
54,HIP12114,K Dwarf (V),HD16160_UVE_1,HIP12114_UVES_1,HD16160,-,UVES,373,42000,45990,...,0.105,86,-0.033,0.043,22,-0.197,0.058,85,6,-


In [ ]:
#!/usr/bin/env python3
"""
Download reduced (Phase 3) ESO spectra for UVES, HARPS, NIRPS given an object identifier.

Uses:
  - Name resolution -> SkyCoord (Sesame) with SIMBAD fallback
  - ESO TAP (tap_obs) -> ivoa.ObsCore query to find spectra (calib_level >= 2)
  - astroquery.eso.Eso.retrieve_data() to download by dp_id

Examples:
  python download_eso_spectra.py "HD 224221" -r 3
  python download_eso_spectra.py "J0418-53" --radius-arcsec 5 --outdir eso_spectra
  python download_eso_spectra.py "HIP 65426" --no-resolve --target-like
"""

from __future__ import annotations

import argparse
import sys
from pathlib import Path

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table

import pyvo
from astroquery.eso import Eso
from astroquery.simbad import Simbad


TAP_OBS_URL = "https://archive.eso.org/tap_obs"
INSTRUMENTS = ("UVES", "HARPS", "NIRPS")


def resolve_name(name: str) -> SkyCoord:
    """Resolve an object name to ICRS coordinates."""
    # 1) Try Sesame via astropy
    try:
        return SkyCoord.from_name(name)
    except Exception:
        pass

    # 2) Fallback to SIMBAD (more explicit control)
    try:
        sim = Simbad()
        sim.add_votable_fields("ra(d)", "dec(d)")
        res = sim.query_object(name)
        if res is None or len(res) == 0:
            raise RuntimeError("Simbad returned no match.")
        return SkyCoord(res["RA_d"][0] * u.deg, res["DEC_d"][0] * u.deg, frame="icrs")
    except Exception as e:
        raise RuntimeError(f"Failed to resolve name '{name}': {e}") from e


def tap_columns(service: pyvo.dal.TAPService) -> set[str]:
    """
    Fetch available column names for ivoa.ObsCore in a pyvo-version-safe way.
    """
    tables = service.tables

    # pyvo >= 1.5: tables.tables is the actual mapping
    if hasattr(tables, "tables"):
        table_map = tables.tables
    else:
        # very old pyvo fallback
        table_map = dict(tables)

    # Try common ObsCore keys
    obscore = (
        table_map.get("ivoa.ObsCore")
        or table_map.get("IVOa.ObsCore")
        or table_map.get("obscore")
    )

    # Final fallback: search by suffix
    if obscore is None:
        for name, table in table_map.items():
            if name.lower().endswith("obscore"):
                obscore = table
                break

    if obscore is None:
        raise RuntimeError(
            "Could not find ivoa.ObsCore table in ESO TAP service. "
            f"Available tables: {list(table_map.keys())[:10]} ..."
        )

    return {col.name for col in obscore.columns}



def pick_instrument_column(cols: set[str]) -> str:
    """Choose the most likely instrument column name."""
    # Common ObsCore column is instrument_name; some deployments use instrument.
    for candidate in ("instrument_name", "instrument", "instrum_name"):
        if candidate in cols:
            return candidate
    raise RuntimeError(
        f"Could not find an instrument column in ObsCore. Available columns include: "
        f"{', '.join(sorted(list(cols))[:40])} ..."
    )


def query_obscore_by_cone(
    target: SkyCoord,
    radius: u.Quantity,
    instruments: tuple[str, ...] = INSTRUMENTS,
    maxrows: int = 20000,
) -> Table:
    """Query ESO tap_obs ivoa.ObsCore for reduced spectra in a cone around target."""
    service = pyvo.dal.TAPService(TAP_OBS_URL)
    cols = tap_columns(service)
    inst_col = pick_instrument_column(cols)

    # Columns we try to request (only those that exist).
    want = [
        "obs_publisher_did",
        "access_url",
        "target_name",
        inst_col,
        "s_ra",
        "s_dec",
        "t_exptime",
        "em_min",
        "em_max",
        "calib_level",
        "obs_collection",
    ]

    select_cols = [c for c in want if c in cols]

    ra_deg = float(target.icrs.ra.deg)
    dec_deg = float(target.icrs.dec.deg)
    rad_deg = float(radius.to_value(u.deg))

    inst_list = ", ".join(f"'{x}'" for x in instruments)

    # Reduced spectra: dataproduct_type='spectrum' and calib_level >= 2
    # Spatial constraint: CONTAINS(point, circle)
    adql = f"""
    SELECT {", ".join(select_cols)}
    FROM ivoa.ObsCore
    WHERE dataproduct_type='spectrum'
      AND calib_level >= 2
      AND {inst_col} IN ({inst_list})
      AND 1 = CONTAINS(
            POINT('ICRS', s_ra, s_dec),
            CIRCLE('ICRS', {ra_deg}, {dec_deg}, {rad_deg})
          )
    """

    job = service.run_async(adql, maxrec=maxrows)
    return job.to_table()


def query_obscore_by_target_like(
    identifier: str,
    instruments: tuple[str, ...] = INSTRUMENTS,
    maxrows: int = 20000,
) -> Table:
    """Fallback query: match target_name with LIKE if name resolution isn't wanted/possible."""
    service = pyvo.dal.TAPService(TAP_OBS_URL)
    cols = tap_columns(service)
    inst_col = pick_instrument_column(cols)

    want = ["dp_id", "target_name", inst_col, "t_exptime", "em_min", "em_max", "calib_level"]
    select_cols = [c for c in want if c in cols]

    inst_list = ", ".join(f"'{x}'" for x in instruments)
    safe = identifier.replace("'", "''")

    adql = f"""
    SELECT {", ".join(select_cols)}
    FROM ivoa.ObsCore
    WHERE dataproduct_type='spectrum'
      AND calib_level >= 2
      AND {inst_col} IN ({inst_list})
      AND (target_name LIKE '%{safe}%')
    """

    job = service.run_async(adql, maxrec=maxrows)
    return job.to_table()


def download_dpids(dp_ids: list[str], outdir: Path, unzip: bool = True) -> list[str]:
    """Download dp_id products via astroquery.eso."""
    outdir.mkdir(parents=True, exist_ok=True)
    eso = Eso()
    # Public data: no login required; if you need proprietary, call eso.login(username=...)
    files = eso.retrieve_data(dp_ids, destination=str(outdir), unzip=unzip)
    # astroquery may return a single string or a list
    if isinstance(files, str):
        return [files]
    return list(files)


def normalize_eso_ids(tbl):
    """
    Ensure we have a usable 'dp_id' column for ESO downloads.
    Priority:
      1) obs_publisher_did  (preferred)
      2) publisher_did
      3) access_url (fallback)
    """
    if "obs_publisher_did" in tbl.colnames:
        tbl["dp_id"] = tbl["obs_publisher_did"]
    elif "publisher_did" in tbl.colnames:
        tbl["dp_id"] = tbl["publisher_did"]
    elif "access_url" in tbl.colnames:
        tbl["dp_id"] = tbl["access_url"]
    else:
        raise RuntimeError(
            f"No usable ESO dataset identifier found. Columns: {tbl.colnames}"
        )
    return tbl



def main() -> int:
    ap = argparse.ArgumentParser()
    ap.add_argument("identifier", help="Object identifier, e.g. 'HD 224221', 'J0418-53', 'HIP 65426'")
    ap.add_argument("-r", "--radius-arcsec", type=float, default=3.0, help="Cone radius in arcsec (default: 3)")
    ap.add_argument("--outdir", type=Path, default=Path("eso_reduced_spectra"), help="Output directory")
    ap.add_argument("--no-resolve", action="store_true", help="Do not resolve name; use target_name LIKE fallback")
    ap.add_argument("--target-like", action="store_true", help="Also do a target_name LIKE query (in addition to cone)")
    ap.add_argument("--maxrows", type=int, default=20000, help="Max rows to fetch from TAP")
    ap.add_argument("--dry-run", action="store_true", help="Only list dp_ids; do not download")
    ap.add_argument("--no-unzip", action="store_true", help="Do not uncompress downloaded files")
    args = ap.parse_args()

    radius = args.radius_arcsec * u.arcsec

    tables = []

    if not args.no_resolve:
        coord = resolve_name(args.identifier)
        print(f"[resolve] {args.identifier} -> RA={coord.ra.deg:.6f} deg, Dec={coord.dec.deg:.6f} deg")
        tbl_cone = query_obscore_by_cone(coord, radius, maxrows=args.maxrows)
        print(f"[tap] cone query returned {len(tbl_cone)} rows")
        tables.append(tbl_cone)

    if args.no_resolve or args.target_like:
        tbl_like = query_obscore_by_target_like(args.identifier, maxrows=args.maxrows)
        print(f"[tap] target_name LIKE query returned {len(tbl_like)} rows")
        tables.append(tbl_like)

    if not tables:
        print("No queries executed (unexpected).", file=sys.stderr)
        return 2

    tbl = Table.vstack([t for t in tables if t is not None and len(t) > 0], metadata_conflicts="silent") if any(len(t) > 0 for t in tables) else Table()
    if len(tbl) == 0:
        print("[result] no matching reduced spectra found.")
        return 0

    # Deduplicate dp_id
    if "dp_id" not in tbl.colnames:
        # Some services might use DP.ID (rare). Try to recover.
        for alt in ("DP.ID", "dp_id", "dpid"):
            if alt in tbl.colnames:
                tbl["dp_id"] = tbl[alt]
                break
    if "dp_id" not in tbl.colnames:
        print(f"[error] Could not find dp_id column in results. Columns: {tbl.colnames}", file=sys.stderr)
        return 3

    dp_ids = sorted({str(x) for x in tbl["dp_id"] if x is not None and str(x).strip() != ""})
    print(f"[result] unique dp_id: {len(dp_ids)}")
    for d in dp_ids[:30]:
        print("  ", d)
    if len(dp_ids) > 30:
        print(f"  ... (+{len(dp_ids) - 30} more)")

    if args.dry_run:
        return 0

    files = download_dpids(dp_ids, args.outdir, unzip=(not args.no_unzip))
    print(f"[download] wrote {len(files)} files into: {args.outdir.resolve()}")
    return 0

In [8]:
coord = resolve_name("HD215665")
tbl_cone = query_obscore_by_cone(coord, 2.0*u.arcsec)
tables = [tbl_cone]
tbl = Table.vstack([t for t in tables if t is not None and len(t) > 0], metadata_conflicts="silent") if any(len(t) > 0 for t in tables) else Table()

if "dp_id" not in tbl.colnames:
    # Some services might use DP.ID (rare). Try to recover.
    for alt in ("DP.ID", "dp_id", "dpid"):
        if alt in tbl.colnames:
            tbl["dp_id"] = tbl[alt]
            break
if "dp_id" not in tbl.colnames:
    print(f"[error] Could not find dp_id column in results. Columns: {tbl.colnames}", file=sys.stderr)
    
dp_ids = sorted({str(x) for x in tbl["dp_id"] if x is not None and str(x).strip() != ""})

files = download_dpids(dp_ids, "data/test_harps")

[error] Could not find dp_id column in results. Columns: []


KeyError: 'dp_id'